In [ ]:
!pip install -q efficientnet >> /dev/null

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
import pandas as pd, numpy as np, random,os, shutil
from glob import glob
from kaggle_datasets import KaggleDatasets
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
#import efficientnet.tfkeras as efn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
print('tf:',tf.__version__)
import math

In [ ]:
DEVICE="TPU"
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path('siimcovidtfrecords')
files_train = np.sort(np.array(tf.io.gfile.glob(GCS_PATH+'/train*.tfrec')))
files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH+'/test*.tfrec')))
num_train_files = len(files_train)
num_test_files  = len(files_test)
print('train_files:',num_train_files)
print('test_files:',num_test_files)

In [ ]:
traindf = pd.read_csv('../input/siimcovidtfrecords/train.csv')
testdf = pd.read_csv('../input/siimcovidtfrecords/test.csv')
names = traindf.classLabel.to_list()
labels = traindf['class'].to_list()
names2labels = dict(set(zip(names,labels)))
labels2names = {v:k for k, v in names2labels.items()}

In [ ]:
FOLDS=5
DEVICE="TPU"
IMG_SIZES = [[512,512]]*FOLDS
BATCH_SIZES = [32]*FOLDS
BATCH_SIZE = 32
EPOCHS      = [12]*FOLDS
PROBABILITY = 0.75
CT          = 8
SZ          = 0.08
SEED=42

In [ ]:
def dropout(image,DIM=IMG_SIZES[0], PROBABILITY = 0.6, CT = 5, SZ = 0.1):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image with CT squares of side size SZ*DIM removed
    
    # DO DROPOUT WITH PROBABILITY DEFINED ABOVE
    P = tf.cast( tf.random.uniform([],0,1)<PROBABILITY, tf.int32)
    if (P==0)|(CT==0)|(SZ==0): 
        return image
    
    for k in range(CT):
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM[1]),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM[0]),tf.int32)
        # COMPUTE SQUARE 
        WIDTH = tf.cast( SZ*min(DIM),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM[0],y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM[1],x+WIDTH//2)
        # DROPOUT IMAGE
        one = image[ya:yb,0:xa,:]
        two = tf.zeros([yb-ya,xb-xa,3], dtype = image.dtype) 
        three = image[ya:yb,xb:DIM[1],:]
        middle = tf.concat([one,two,three],axis=1)
        image = tf.concat([image[0:ya,:,:],middle,image[yb:DIM[0],:,:]],axis=0) 

    image = tf.reshape(image,[*DIM,3])
    return image

In [ ]:
#https://medium.com/mlait/affine-transformation-image-processing-in-tensorflow-part-1-df96256928a
#ref:https://en.wikipedia.org/wiki/Rotation_matrix
#ref:https://en.wikipedia.org/wiki/Shear_matrix
#ref:https://en.wikipedia.org/wiki/Scaling_(geometry)
def getTransformMat(rotation, shear, yzoom, xzoom, yshift, xshift):
        
    rot = math.pi * rotation/180.
    shear    = math.pi * shear/ 180.
    
    cos1   = tf.math.cos(rot)
    sin1   = tf.math.sin(rot)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotMat = tf.reshape(tf.concat([[cos1,   sin1,   zero, -sin1,  cos1,   zero, zero, zero, one]],axis=0), [3,3])
    cos2 = tf.math.cos(shear)
    sin2 = tf.math.sin(shear)    
    shearMat = tf.reshape(tf.concat([[one,  sin2,   zero, zero, cos2,   zero, zero, zero, one]],axis=0), [3,3])        
    zoomMat = tf.reshape(tf.concat([[one/yzoom, zero, zero, zero, one/xzoom, zero,zero, zero, one]],axis=0), [3,3])    
    shiftMat = tf.reshape(tf.concat([[one,  zero, yshift, zero, one,  xshift, zero, zero, one]],axis=0), [3,3])
    
    return K.dot(K.dot(rotMat, shearMat), K.dot(zoomMat,shiftMat))

def transform(image, DIM=IMG_SIZES[0]):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    
    # fixed for non-square image thanks to Chris Deotte
    
    if DIM[0]!=DIM[1]:
        pad = (DIM[0]-DIM[1])//2
        image = tf.pad(image, [[0, 0], [pad, pad+1],[0, 0]])
        
    NEW_DIM = DIM[0]
    
    XDIM = NEW_DIM%2 #fix for size 331
    
    rot = 0.0 * tf.random.normal([1], dtype='float32')
    shr = 2.0 * tf.random.normal([1], dtype='float32') 
    yzoom = 1.0 + tf.random.normal([1], dtype='float32') / 8.0
    xzoom = 1.0 + tf.random.normal([1], dtype='float32') / 8.0
    yshift = 8.0 * tf.random.normal([1], dtype='float32') 
    xshift = 8.0 * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = getTransformMat(rot,shr,yzoom,xzoom,yshift,xshift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(NEW_DIM//2, -NEW_DIM//2,-1), NEW_DIM)
    y   = tf.tile(tf.range(-NEW_DIM//2, NEW_DIM//2), [NEW_DIM])
    z   = tf.ones([NEW_DIM*NEW_DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -NEW_DIM//2+XDIM+1, NEW_DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([NEW_DIM//2-idx2[0,], NEW_DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
    
    if DIM[0]!=DIM[1]:
        image = tf.reshape(d,[NEW_DIM, NEW_DIM,3])
        image = image[:, pad:DIM[1]+pad,:]
    image = tf.reshape(image, [*DIM, 3])
        
    return image

In [ ]:
def readTrain(example):
    tfrec_format = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_id': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], tf.one_hot(example['target'], 4)


def readTest(example, return_image_id):
    tfrec_format = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_id': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_id'] if return_image_id else 0

In [ ]:
def prepImage(img, augment=True, dim=IMG_SIZES[0]): 
    sat  = (0.7, 1.3)
    cont = (0.8, 1.2)
    bri  =  0.1
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    img = img/255.0
    
    if augment:
        img = transform(img,DIM=dim)
        img = tf.image.random_flip_left_right(img)
        #img = tf.image.random_hue(img, 0.01)
        img = tf.image.random_saturation(img, sat[0], sat[1])
        img = tf.image.random_contrast(img, cont[0], cont[1])
        img = tf.image.random_brightness(img, bri)     
                      
    img = tf.reshape(img, [*dim, 3])
            
    return img

In [ ]:
def count_data_items(fileids):
    n = [int(re.compile(r"-([0-9]*)\.").search(fileid).group(1)) 
         for fileid in fileids]
    return np.sum(n)

In [ ]:
def get_dataset(files, augment = False, shuffle = False, repeat = False, 
                labeled=True, return_image_ids=True, batch_size=16, dim=IMG_SIZES[0]):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*2, seed=SEED)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        ds = ds.map(readTrain, num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: readTest(example, return_image_ids), 
                    num_parallel_calls=AUTO)      
    
    ds = ds.map(lambda img, imgid_or_label: (prepImage(img, augment=augment, dim=dim), 
                                               imgid_or_label), 
                num_parallel_calls=AUTO)
    if labeled and augment:
        ds = ds.map(lambda img, label: (dropout(img, DIM=dim, PROBABILITY = PROBABILITY, CT = CT, SZ = SZ), label),
                    num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds

In [ ]:
def display_batch(batch, size=2):
    imgs, tars = batch
    plt.figure(figsize=(size*5, 5))
    for img_idx in range(size):
        plt.subplot(1, size, img_idx+1)
        plt.title(f'class: {labels2names[tars[img_idx].numpy().argmax()]}', fontsize=15)
        plt.imshow(imgs[img_idx,:, :, :])
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    plt.show() 

In [ ]:
fold = 0
ds = get_dataset(files_train, augment=True, shuffle=False, repeat=True,labeled=True,return_image_ids=True,
                dim=IMG_SIZES[fold], batch_size = BATCH_SIZES[fold])
ds = ds.unbatch().batch(20)
batch = next(iter(ds))
display_batch(batch, 5);

# Training

In [ ]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

def build_model(dim=IMG_SIZES[0], ef=0):
    inp = tf.keras.layers.Input(shape=(*dim,3))
    base = EFNS[ef](input_shape=(*dim,3),weights='imagenet',include_top=False)
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    x = tf.keras.layers.Dense(4,activation='softmax')(x)
    model = tf.keras.Model(inputs=inp,outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01)
    auc = tf.keras.metrics.AUC(curve='ROC',
                               multi_label=True)
    acc = tf.keras.metrics.CategoricalAccuracy()
    f1  = tfa.metrics.F1Score(num_classes=4,average='macro',threshold=None)
    model.compile(optimizer=opt,loss=loss,metrics=[auc, acc, f1])
    return model

In [ ]:

def get_lr_callback(batch_size=8, plot=False):
    lr_start   = 0.000005
    lr_max     = 0.00000125 * REPLICAS * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr
    if plot:
        plt.figure(figsize=(10,5))
        plt.plot(np.arange(EPOCHS[0]), [lrfn(epoch) for epoch in np.arange(EPOCHS[0])], marker='o')
        plt.xlabel('epoch'); plt.ylabel('learnig rate')
        plt.title('Learning Rate Scheduler')
        plt.show()

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

_=get_lr_callback(BATCH_SIZES[0], plot=True )

In [ ]:
# USE VERBOSE=0 for silent, VERBOSE=1 for interactive, VERBOSE=2 for commit
VERBOSE      = 0
DISPLAY_PLOT = True

DEVICE = "TPU" #or "GPU"

# USE DIFFERENT SEED FOR DIFFERENT STRATIFIED KFOLD
SEED = 42

# NUMBER OF FOLDS. USE 2, 5, 10
FOLDS = 5

# WHICH IMAGE SIZES TO LOAD EACH FOLD
# CHOOSE 128, 192, 256, 384, 512, 512 
IMG_SIZES = [[512, 512]]*FOLDS

# BATCH SIZE AND EPOCHS
BATCH_SIZES = [16]*FOLDS
EPOCHS      = [12]*FOLDS

# WHICH EFFICIENTNET B? TO USE
EFF_NETS = [7]*FOLDS

# Augmentations
AUGMENT   = True
TRANSFORM = True

# transformations
ROT_    = 0.0
SHR_    = 2.0
HZOOM_  = 8.0
WZOOM_  = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

# Dropout
PROBABILITY = 0.75
CT          = 8
SZ          = 0.08

#bri, contrast
sat  = (0.7, 1.3)
cont = (0.8, 1.2)
bri  =  0.1

# TEST TIME AUGMENTATION STEPS
TTA = 1

In [ ]:
EFF_NETS = [7]*FOLDS
TTA=1
#DISPLAY_PLOT = True
skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)
oof_pred = []; oof_tar = []; oof_val = []; oof_f1 = []; oof_ids = []; oof_folds = [] 
preds = np.zeros((count_data_items(files_test),1))

for fold,(idxT,idxV) in enumerate(skf.split(np.arange(num_train_files))):
    # DISPLAY FOLD INFO
    if DEVICE=='TPU':
        if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    
    # CREATE TRAIN AND VALIDATION SUBSETS
    files_train = tf.io.gfile.glob([GCS_PATH + '/train%.2i*.tfrec'%x for x in idxT])
    np.random.shuffle(files_train);
    files_valid = tf.io.gfile.glob([GCS_PATH + '/train%.2i*.tfrec'%x for x in idxV])
    files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')))
    print('#'*25); print('#### FOLD',fold+1)
    print('#### Image Size: (%i, %i) | model: %s | batch_size %i'%
          (IMG_SIZES[fold][0],IMG_SIZES[fold][1],EFNS[EFF_NETS[fold]].__name__,BATCH_SIZES[fold]*REPLICAS))
    train_images = count_data_items(files_train)
    val_images   = count_data_items(files_valid)
    print('#### Training: %i | Validation: %i'%(train_images, val_images))
    
    # BUILD MODEL
    K.clear_session()
    with strategy.scope():
        model = build_model(dim=IMG_SIZES[fold],ef=EFF_NETS[fold])
    print('#'*25)   
    # SAVE BEST MODEL EACH FOLD
    sv = tf.keras.callbacks.ModelCheckpoint(
        'fold-%i.h5'%fold, monitor='val_auc', verbose=2, save_best_only=True,
        save_weights_only=True, mode='max', save_freq='epoch')
   
    # TRAIN
    print('Training...')
    history = model.fit(
        get_dataset(files_train, augment=True, shuffle=True, repeat=True,
                dim=IMG_SIZES[fold], batch_size = BATCH_SIZES[fold]), 
        epochs=EPOCHS[fold], 
        callbacks = [sv,get_lr_callback(BATCH_SIZES[fold])], 
        steps_per_epoch=count_data_items(files_train)/BATCH_SIZES[fold]//REPLICAS,
        validation_data=get_dataset(files_valid,augment=False,shuffle=False,repeat=False,dim=IMG_SIZES[fold]),
        verbose = False,
    )
    
    # Loading best model for inference
    print('Loading best model...')
    model.load_weights('fold-%i.h5'%fold)  
    
    # PREDICT OOF USING TTA
    print('Predicting OOF with TTA...')
    ds_valid = get_dataset(files_valid,labeled=False,return_image_ids=False,augment=True if TTA>1 else False,
            repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*2)
    ct_valid = count_data_items(files_valid); STEPS = TTA * ct_valid/BATCH_SIZES[fold]/2/REPLICAS
    pred = model.predict(ds_valid,steps=STEPS,verbose=2)[:TTA*ct_valid,] 
    oof_pred.append( np.mean(pred.reshape((ct_valid,4,TTA),order='F'),axis=-1) )                 
    
    # GET OOF TARGETS AND idS
    ds_valid = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold],
            labeled=True, return_image_ids=True)
    oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )
    oof_folds.append( np.ones_like(oof_tar[-1],dtype='int8')*fold )
    ds = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold],
                labeled=False, return_image_ids=True)
    oof_ids.append( np.array([img_id.numpy().decode("utf-8") for img, img_id in iter(ds.unbatch())]))
        
    # REPORT RESULTS
    auc = roc_auc_score(oof_tar[-1],oof_pred[-1], average='macro')
    oof_val.append(np.max( history.history['val_auc'] ))
    print('#### FOLD %i OOF AUC without TTA = %.3f, with TTA = %.3f'%(fold+1,oof_val[-1],auc))
    fig, (ax1, ax2) = plt.subplots(2,1)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    #plt.figure(figsize=(15,5))
    ax1.plot(np.arange(len(history.history['auc'])),history.history['auc'],'-o',label='Train auc',color='#ff7f0e')
    ax1.plot(np.arange(len(history.history['auc'])),history.history['val_auc'],'-o',label='Val auc',color='#1f77b4')
    x = np.argmax(history.history['val_auc'] ); y = np.max(history.history['val_auc'] )
    xdist = ax1.get_xlim()[1] - ax1.get_xlim()[0]; ydist = ax1.get_ylim()[1] - ax1.get_ylim()[0]
    ax1.scatter(x,y,s=200,color='#1f77b4'); ax1.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
    ax1.set_ylabel('auc',size=14); ax1.set_xlabel('Epoch',size=14)
    ax1.set_title('Epoch vs Accuracy')
    #ax1.legend(loc=2)
    ax2.plot(np.arange(len(history.history['auc'])),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
    ax2.plot(np.arange(len(history.history['auc'])),history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
    x = np.argmin(history.history['val_loss'] ); y = np.min(history.history['val_loss'] )
    ydist = ax2.get_ylim()[1] - ax2.get_ylim()[0]
    ax2.scatter(x,y,s=200,color='#d62728'); ax2.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    ax2.set_ylabel('Loss',size=14)
    ax2.set_xlabel('Epoch',size=14)
    ax2.set_title('Epoch vs Loss')
    ax2.legend(loc=3)
    fig.savefig(f'fig{fold}.png')
    

In [ ]:
# COMPUTE OVERALL OOF AUC
oof = np.concatenate(oof_pred); true = np.concatenate(oof_tar);
ids = np.concatenate(oof_ids); folds = np.concatenate(oof_folds)
auc = roc_auc_score(true,oof, average='macro')
print('Overall OOF AUC with TTA = %.3f'%auc)

# SAVE OOF TO DISK
columns = ['image_id', 'fold']+[f'{idx}_true' for idx in range(4)] + [f'{idx}_pred' for idx in range(4)]
df_oof = pd.DataFrame(np.concatenate([ids[:,None], folds[:, 0:1], true, oof], axis=1), columns=columns)
df_oof.to_csv('oof.csv',index=False)
df_oof.head()